In [1]:
import pandas as pd

In [2]:
question_answer = pd.read_csv('csv/Posts.csv')

In [3]:
question_answer.set_index('Id', inplace=True)

In [4]:
answers = question_answer[question_answer['PostTypeId'] == 2]
questions = question_answer[question_answer['PostTypeId'] == 1]


In [5]:
answerer = answers.groupby('OwnerUserId')['ParentId'].apply(list).to_dict()

In [6]:
question_tags = questions['Tags'].apply(lambda x: list(filter(lambda x: x != '', x.split('|'))))


In [7]:
count_answers_per_user = answers['OwnerUserId'].value_counts().to_dict()

In [8]:
count_answers_per_user

{9113.0: 2839,
 177980.0: 2326,
 1204.0: 2043,
 123788.0: 1697,
 131624.0: 1612,
 60357.0: 1590,
 3965.0: 1551,
 209774.0: 1510,
 5099.0: 1418,
 6605.0: 1240,
 51654.0: 1098,
 7422.0: 1029,
 855.0: 994,
 4.0: 978,
 106566.0: 895,
 22685.0: 895,
 1352.0: 883,
 63202.0: 759,
 199871.0: 747,
 118878.0: 738,
 172693.0: 702,
 6509.0: 665,
 209665.0: 638,
 209331.0: 627,
 56655.0: 623,
 61852.0: 526,
 16929.0: 506,
 115084.0: 503,
 935.0: 501,
 319783.0: 499,
 13156.0: 498,
 897.0: 496,
 6967.0: 469,
 40065.0: 460,
 5834.0: 457,
 8553.0: 447,
 20011.0: 446,
 105684.0: 444,
 11982.0: 433,
 902.0: 429,
 12828.0: 420,
 3762.0: 419,
 235743.0: 416,
 30278.0: 409,
 34148.0: 407,
 25768.0: 403,
 4327.0: 398,
 18930.0: 395,
 232369.0: 389,
 222996.0: 386,
 66037.0: 380,
 41811.0: 379,
 906.0: 375,
 57164.0: 371,
 6586.0: 370,
 13181.0: 369,
 25476.0: 365,
 4854.0: 357,
 620.0: 354,
 28696.0: 353,
 1093.0: 350,
 14221.0: 332,
 41381.0: 316,
 667.0: 316,
 116777.0: 315,
 6617.0: 310,
 9007.0: 301,
 1

In [9]:
count_tag = dict()
for tags in question_tags:
  for tag in tags:
    if tag not in count_tag:
      count_tag[tag] = 0
    count_tag[tag] += 1

In [10]:
count_tag

{'comments': 199,
 'anti-patterns': 237,
 'business': 127,
 'project-management': 925,
 'development-process': 736,
 'productivity': 357,
 'time-management': 87,
 'programming-languages': 1403,
 'dynamic-typing': 103,
 'static-typing': 86,
 'type-systems': 171,
 'team': 225,
 'web-development': 1557,
 'waterfall': 49,
 'quotations': 6,
 'communication': 232,
 'skills': 162,
 'functional-programming': 737,
 'coding-style': 1056,
 'indentation': 27,
 'scope-creep': 9,
 'interview': 290,
 'hiring': 58,
 'billing': 20,
 'freelancing': 206,
 'pricing': 55,
 'twitter': 21,
 'social-networks': 35,
 'books': 125,
 'language-agnostic': 549,
 'concurrency': 374,
 'erlang': 40,
 'actor-model': 46,
 'usage': 8,
 'performance': 835,
 'code-quality': 653,
 'test-coverage': 77,
 'tdd': 668,
 'linux': 299,
 'kernel': 37,
 'agile': 1140,
 'solo-development': 62,
 'terminology': 764,
 'history': 348,
 'variables': 205,
 'learning': 356,
 'cobol': 17,
 'design-patterns': 4449,
 'singleton': 142,
 'java':

In [11]:
answerer_tag = pd.DataFrame()

In [12]:
user_tag_count = {}
for user in answerer:
  if count_answers_per_user[user] < 20:
    continue
  tag_count = {}
  for question in answerer[user]:
    for tag in question_tags[question]:
      if count_tag[tag] < 20:
        continue
      if tag not in tag_count:
        tag_count[tag] = 0
      tag_count[tag] += 1
  user_tag_count[user] = tag_count

In [13]:
answerer_tag = pd.DataFrame.from_dict(user_tag_count, orient='index')

In [14]:
answerer_tag.fillna(0, inplace=True)

In [15]:
answerer_tag

,comments,anti-patterns,books,language-agnostic,performance,productivity,development-process,project-management,estimation,time-management,...,golang,import,geospatial,combinatorics,elasticsearch,factory-pattern,proxy,strategy-pattern,serverless,change-management
4.0,13.0,2.0,10.0,8.0,8.0,8.0,55.0,61.0,29.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11.0,1.0,1.0,2.0,0.0,0.0,4.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15.0,1.0,1.0,1.0,3.0,1.0,4.0,4.0,4.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17.0,1.0,1.0,0.0,1.0,2.0,5.0,2.0,3.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66.0,1.0,0.0,1.0,1.0,0.0,3.0,1.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226041.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
236092.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
314234.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
